# Introduction

This notebook finished the tasks from the [mini project document](LMM.pdf). 

There is a a dataset $\{ \mathbf{y}, \mathbf{X}, \mathbf{Z} \}$ with $n$ samples, 
where $\mathbf{y} \in \mathbb{R}^n$ is the vector of response variable, 
$\mathbf{X} \in \mathbb{R}^{n \times p}$ is the matrix of $p$ independent variables, 
and $\mathbf{Z} \in \mathbb{R}^{n \times c}$ is another matrix of $c$ variables. The notebook aimed to implement an Expectation-Maximization (EM) algorithm to find the best set of parameter $\Theta = \{\omega, \sigma^2_{\beta}, \sigma^2_{e}\}$ for a **linear mixed model** (LMM) 
$$
y = \mathbf{Z}\boldsymbol{\omega} + \mathbf{X}\boldsymbol{\beta} + \mathbf{e},
\qquad 
\boldsymbol{\beta}\sim \mathcal N(\mathbf{0},\,\sigma_\beta^2 \mathbf{I}_p), \quad
\mathbf{e}\sim \mathcal N(\mathbf{0},\,\sigma_e^2 \mathbf{I}_n).
$$ 


The implemented algorithm was applied to the [dataset](XYZ_MoM.txt).

Besides, it also demonstrated the implementation of the mean-field varaition inference (MFVI). A comparision between the MFVI and EM was made.

# 0. Load packages

In [1]:
import numpy as np

# 1. Implementation of the EM algorithm

This step implemented the EM algorithm for a LMM. 

**Return at each iteration:**
- the marginal log-likelihood, 
- the current estimate of $\Theta=\{\boldsymbol{\omega},\sigma_\beta^2,\sigma_e^2\}$,
- and the posterior mean of $\boldsymbol{\beta}$.

## 1.1 Model setup and notations

- $y\in\mathbb R^n$: response vector  
- $\mathbf{Z}\in\mathbb R^{n\times c}$: fixed-effects design  
- $\mathbf{X}\in\mathbb R^{n\times p}$: random-effects design  
- $\boldsymbol{\omega}\in\mathbb R^c$: fixed effects
- $\boldsymbol{\beta}\in\mathbb R^p$: random effects
- $\mathbf{e}\in\mathbb R^n$: independent noise term 
- Unknown parameters: $\Theta=\{\boldsymbol{\omega},\sigma_\beta^2,\sigma_e^2\}$.  
- Latent variable: $\boldsymbol{\beta}\in\mathbb R^p$.

We treat $\boldsymbol{\beta}$ as **latent** because it is not observed; its prior is
$\boldsymbol{\beta}\sim\mathcal N(\mathbf{0},\sigma_\beta^2 \mathbf{I}_p)$.

# 1.2 Marginal likelihood (integrating out $\beta$)


**Conditionally on $\boldsymbol{\beta}$:**

Given fixed $\mathbf{Z}$, $\boldsymbol{\omega}$, and $\mathbf{X}$, when we conditioned on $\boldsymbol{\beta}$, $\mathbf{Z}\boldsymbol{\omega} + \mathbf{X}\boldsymbol{\beta}$ can be regarded as a constant. $y \mid \boldsymbol{\beta} = (\mathbf{Z}\boldsymbol{\omega} + \mathbf{X}\boldsymbol{\beta}) + \mathbf{e}$ is a _constant+gaussian noise_.

$$
y \mid \boldsymbol{\beta} \sim \mathcal N\!\big(\mathbf{Z}\boldsymbol{\omega} + \mathbf{X}\boldsymbol{\beta},\ \sigma_e^2 \mathbf{I}_n\big).
$$

**Integrating out $\boldsymbol{\beta}$ gives:**

- **Expectation:**
$$
\mathbb{E}[y] 
= \mathbb{E}_\beta\big[\mathbb{E}[y \mid \beta]\big] 
= \mathbb{E}_\beta[Z\omega + X\beta] 
= Z\omega + X \mathbb{E}[\beta] 
= Z\omega,
$$
because $$\beta \sim \mathcal N(0,\sigma_\beta^2 I_p)$$ has mean 0.


- **Covariance:**
$$
\operatorname{Cov}(y) 
= \mathbb{E}_\beta\!\big[\operatorname{Cov}(y \mid \beta)\big] 
+ \operatorname{Cov}_\beta\!\big(\mathbb{E}[y \mid \beta]\big).
$$

Here,
$$
\operatorname{Cov}(y \mid \beta) = \sigma_e^2 I_n, 
\qquad 
\mathbb{E}[y \mid \beta] = Z\omega + X\beta.
$$

Thus,
$$
\operatorname{Cov}(y) 
= \sigma_e^2 I_n + \operatorname{Cov}(X\beta)
= \sigma_e^2 I_n + X\,\operatorname{Cov}(\beta)\,X^\top
= \sigma_e^2 I_n + \sigma_\beta^2 XX^\top.
$$

$$
y \sim \mathcal N\!\big(\mathbf{Z}\boldsymbol{\omega},\ \boldsymbol{\Sigma}_y\big),
\qquad
\boldsymbol{\Sigma}_y = \sigma_e^2 \mathbf{I}_n + \sigma_\beta^2 \mathbf{X}\mathbf{X}^\top.
$$

**The marginal log-likelihood is:**
$$
\log p(y\mid\Theta)
= -\tfrac{n}{2}\log(2\pi)\ -\ \tfrac{1}{2}\log\lvert \boldsymbol{\Sigma}_y\rvert\ -\
\tfrac{1}{2}(y-\mathbf{Z}\boldsymbol{\omega})^\top \boldsymbol{\Sigma}_y^{-1}(y-\mathbf{Z}\boldsymbol{\omega}).
$$

This comes from the standard density of the multivariate Gaussian distribution:

$$
p(y) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}} 
\exp\!\left(-\tfrac{1}{2}(y-\mu)^\top \Sigma^{-1}(y-\mu)\right).
$$

Apply it here with $\mu = Z\omega$, $\Sigma = \Sigma_y$.

Taking the logarithm, we obtain three terms:

1. $-\tfrac{n}{2}\log(2\pi)$: constant term.  
2. $-\tfrac{1}{2}\log|\Sigma_y|$: penalty for large covariance volume.  
3. $-\tfrac{1}{2}(y - Z\omega)^\top \Sigma_y^{-1}(y - Z\omega)$: Mahalanobis distance, i.e. the discrepancy between sample and mean weighted by the covariance.  


We will evaluate this at each EM iteration.


## 1.3 E-step

In the E-step, we compute the expected value of the complete-data log-likelihood 
with respect to the posterior distribution of the latent variable $\beta$
under the current parameter estimates $\Theta^{(t)}$:

$$
Q(\Theta \mid \Theta^{(t)}) = \mathbb E_{\boldsymbol{\beta}\mid y,\Theta^{(t)}}
\big[\log p(y,\boldsymbol{\beta}\mid \Theta)\big].
$$

The _posterior of $\boldsymbol{\beta}$ under current parameters ($\Theta^{(t)}$) is:_
$$
\boldsymbol{\beta}\mid y,\Theta^{(t)} \sim \mathcal N\!\big(\mathbf{m}^{(t)},\ \mathbf{S}^{(t)}\big)
$$


**Notes:**
- The E-step **does not optimize** any parameters.
- It **computes the expectation** of the complete-data log-likelihood using the 
posterior of $\beta$ under current parameters $\Theta^{(t)}$.
- This expected value $Q(\Theta \mid \Theta^{(t)})$ 
becomes the objective function for the M-step.

### 1.3.1 Complete-data log-likelihood

The complete-data consists of the observed data $y$ and the latent variables $\beta$.  
By the model,
$$
p(y,\beta \mid \Theta) = p(y \mid \beta, \Theta)\, p(\beta \mid \Theta).
$$

- **Conditional likelihood:**
$$
y \mid \beta \sim \mathcal N(Z\omega + X\beta,\ \sigma_e^2 I_n).
$$

- **Prior:**
$$
\beta \sim \mathcal N(0,\ \sigma_\beta^2 I_p).
$$

Thus the complete-data log-likelihood (ignoring additive constants) is:
$$
\log p(y,\beta \mid \Theta)
= -\frac{n}{2}\log\sigma_e^2
  -\frac{1}{2\sigma_e^2}\|y - Z\omega - X\beta\|^2
  -\frac{p}{2}\log\sigma_\beta^2
  -\frac{1}{2\sigma_\beta^2}\|\beta\|^2.
$$


### 1.3.2 Posterior of $\beta$

Using Bayes’ theorem:

$$
p(\beta \mid y,\Theta) \propto p(y,\beta \mid \Theta)
= p(y \mid \beta, \Theta)\,p(\beta \mid \Theta).
$$

Taking the logarithm again (up to a constant):

$$
\log p(\beta \mid y,\Theta)
= -\frac{1}{2\sigma_e^2}\|y - Z\omega - X\beta\|^2
  -\frac{1}{2\sigma_\beta^2}\|\beta\|^2 + C.
$$


Expand the first term:

$$
\|y - Z\omega - X\beta\|^2
= (y - Z\omega)^\top(y - Z\omega)
  - 2\beta^\top X^\top(y - Z\omega)
  + \beta^\top X^\top X\beta.
$$

Substitute this into the log posterior:

$$
\log p(\beta \mid y,\Theta)
= -\frac{1}{2}\beta^\top
   \big(\sigma_e^{-2}X^\top X + \sigma_\beta^{-2}I_p\big)\beta
  + \beta^\top
   \big(\sigma_e^{-2}X^\top(y - Z\omega)\big)
  + \text{const}.
$$


Now, we identify the Gaussian form by "completing the square".

The above is a standard multivariate Gaussian log-density of the form:

$$
\log p(\beta \mid y,\Theta)
= -\frac{1}{2}(\beta - m)^\top S^{-1}(\beta - m) + \text{const}.
$$

Matching terms, we can identify:

$$
S^{-1} = \sigma_\beta^{-2}I_p + \sigma_e^{-2}X^\top X,
\qquad
S = \Big(\sigma_\beta^{-2}I_p + \sigma_e^{-2}X^\top X\Big)^{-1},
$$

$$
m = S\big(\sigma_e^{-2}X^\top(y - Z\omega)\big).
$$

Under the current parameter $\Theta^{(t)}$, the posterior distribution of 
$\beta$ is Gaussian:

$$
\beta \mid y,\Theta^{(t)} \sim 
\mathcal N\!\big(m^{(t)},\, S^{(t)}\big),
$$

where

$$
S^{(t)} = 
\Big(\sigma_\beta^{-2\,(t)} I_p 
      + \sigma_e^{-2\,(t)} X^\top X\Big)^{-1},
\qquad
m^{(t)} = 
S^{(t)}\!\Big(\sigma_e^{-2\,(t)} X^\top (y - Z\omega^{(t)})\Big).
$$


### 1.3.3 Compute the expectations

To take the expectation of $\log p(y,\beta \mid \Theta)$,
we only need to compute expectations involving $\beta$:

#### (a) Second moment of $\beta$
$$
\mathbb{E}\|\beta\|^2
= \|m^{(t)}\|^2 + \operatorname{tr}(S^{(t)}).
$$

This follows from the moment identity for a Gaussian:
$\mathbb{E}[xx^\top] = \mu\mu^\top + \Sigma.$

#### (b) Expected squared residual
$$
\mathbb{E}\|y - Z\omega - X\beta\|^2
= \|y - Z\omega - X m^{(t)}\|^2
  + \operatorname{tr}(X S^{(t)} X^\top).
$$

This comes from variance decomposition:
$$
\mathbb{E}\|a - X\beta\|^2 
= \|a - X\mathbb{E}[\beta]\|^2 
+ \operatorname{tr}(X \operatorname{Var}(\beta) X^\top).
$$

### 1.3.4 Substitute into $Q(\Theta \mid \Theta^{(t)})$

Substitute the two expectations into the complete-data log-likelihood:

$$
\boxed{
Q(\Theta \mid \Theta^{(t)}) 
= -\frac{n}{2}\log\sigma_e^2
  -\frac{1}{2\sigma_e^2}
     \Big(\|y - Z\omega - X m^{(t)}\|^2
          + \operatorname{tr}(X S^{(t)} X^\top)\Big)
  -\frac{p}{2}\log\sigma_\beta^2
  -\frac{1}{2\sigma_\beta^2}
     \Big(\|m^{(t)}\|^2
          + \operatorname{tr}(S^{(t)})\Big).
}
$$

## 1.4 M-step 

Given the E-step posterior $\beta \mid y,\Theta^{(t)} \sim \mathcal N(\mathbf{m}^{(t)}, \mathbf{S}^{(t)})$,
the $Q$-function (up to additive constants) is
$$
Q(\Theta\mid\Theta^{(t)}) 
= -\frac{n}{2}\log\sigma_e^{2}
  -\frac{1}{2\sigma_e^{2}}
     \Big(\,\|y-\mathbf{Z}\boldsymbol{\omega}-\mathbf{X}\mathbf{m}^{(t)}\|^2
          + \operatorname{tr}(\mathbf{X}\mathbf{S}^{(t)}\mathbf{X}^\top)\Big)
  -\frac{p}{2}\log\sigma_\beta^{2}
  -\frac{1}{2\sigma_\beta^{2}}
     \Big(\,\|\mathbf{m}^{(t)}\|^2 + \operatorname{tr}(\mathbf{S}^{(t)})\Big).
$$

We maximize $Q$ w.r.t. $\boldsymbol{\omega}$, $\sigma_e^2$, and $\sigma_\beta^2$.

### 1.4.1 Update for $\boldsymbol{\omega}$

**Problem:** minimize the quadratic term in $\boldsymbol{\omega}$:
$$
\min_{\boldsymbol{\omega}} \ \|y-\mathbf{Z}\boldsymbol{\omega}-\mathbf{X}\mathbf{m}^{(t)}\|^2.
$$

**Gradient:** 
$$
\frac{\partial}{\partial \boldsymbol{\omega}}
\|y-\mathbf{Z}\boldsymbol{\omega}-\mathbf{X}\mathbf{m}^{(t)}\|^2
= -2\mathbf{Z}^\top\!\big(y-\mathbf{Z}\boldsymbol{\omega}-\mathbf{X}\mathbf{m}^{(t)}\big).
$$

Set to zero:
$$
\mathbf{Z}^\top\mathbf{Z}\,\boldsymbol{\omega}^{(t+1)}
= \mathbf{Z}^\top\!\big(y - \mathbf{X}\mathbf{m}^{(t)}\big)
\quad\Longrightarrow\quad
\boxed{\;
\boldsymbol{\omega}^{(t+1)} 
= (\mathbf{Z}^\top\mathbf{Z})^{-1}\mathbf{Z}^\top\!\big(y-\mathbf{X}\mathbf{m}^{(t)}\big).
\;}
$$

**Principle:** ordinary least squares (OLS) of the “E-step–denoised response”
$y - \mathbf{X}\mathbf{m}^{(t)}$ onto the fixed-effects design $\mathbf{Z}$.


### 1.4.2 Update for $\sigma_e^2$

Keep only the $\sigma_e^2$-dependent part of $Q$:
$$
Q_e(\sigma_e^2)
= -\frac{n}{2}\log\sigma_e^{2}
  -\frac{1}{2\sigma_e^{2}}
     \underbrace{\Big(\|y-\mathbf{Z}\boldsymbol{\omega}^{(t+1)}-\mathbf{X}\mathbf{m}^{(t)}\|^2
     + \operatorname{tr}(\mathbf{X}\mathbf{S}^{(t)}\mathbf{X}^\top)\Big)}_{=: \ \mathcal{R}_e^{(t)} }.
$$

Differentiate and set to zero:
$$
\frac{\partial Q_e}{\partial \sigma_e^2}
= -\frac{n}{2}\frac{1}{\sigma_e^2}
  + \frac{1}{2}\frac{\mathcal{R}_e^{(t)}}{(\sigma_e^2)^2} = 0
\ \Longrightarrow\
\boxed{\;
\sigma_e^{2\,(t+1)} 
= \frac{\ \|y-\mathbf{Z}\boldsymbol{\omega}^{(t+1)}-\mathbf{X}\mathbf{m}^{(t)}\|^2
     + \operatorname{tr}(\mathbf{X}\mathbf{S}^{(t)}\mathbf{X}^\top)}{n}.
\;}
$$

**Principle:** MLE of a Gaussian variance equals the **average expected squared residual**.


### 1.4.3 Update for $\sigma_\beta^2$

Keep only the $\sigma_\beta^2$-dependent part:
$$
Q_\beta(\sigma_\beta^2)
= -\frac{p}{2}\log\sigma_\beta^{2}
  -\frac{1}{2\sigma_\beta^{2}}
     \underbrace{\Big(\|\mathbf{m}^{(t)}\|^2 + \operatorname{tr}(\mathbf{S}^{(t)})\Big)}_{=: \ \mathcal{R}_\beta^{(t)}}.
$$

Differentiate and set to zero:
$$
\frac{\partial Q_\beta}{\partial \sigma_\beta^2}
= -\frac{p}{2}\frac{1}{\sigma_\beta^2}
  + \frac{1}{2}\frac{\mathcal{R}_\beta^{(t)}}{(\sigma_\beta^2)^2}=0
\ \Longrightarrow\
\boxed{\;
\sigma_\beta^{2\,(t+1)} 
= \frac{\ \|\mathbf{m}^{(t)}\|^2 + \operatorname{tr}(\mathbf{S}^{(t)})\ }{p}.
\;}
$$

**Principle:** the prior variance matches the **average posterior second moment** of $\beta$.